# Translation

## Preparing the data

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4.py", lang1="en", lang2="fr", trust_remote_code=True)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [4]:
split_datasets["validation"] = split_datasets.pop("test")

#### checking  Model Helsinki-NLP/opus-mt-en-fr model

In [5]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [6]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

In [7]:
translator("Default to expanded threads")

# In the dataset, the translations for threads is "fils de discussion", where as the "Helsinki-NLP/opus-mt-en-fr" model translates it to "threads".

[{'translation_text': 'Par défaut pour les threads élargis'}]

In [8]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [9]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

# Another example of this behavior can be seen with the word “plugin,” which isn’t officially a French word but which most native speakers will understand and not bother to translate. In the KDE4 dataset this word has been translated in French into the more official “module d’extension”

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

## Processing the data

In [10]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

# If you are using a multilingual tokenizer such as mBART, mBART-50, or M2M100, you will need to set the language codes of your inputs and targets in the tokenizer by setting tokenizer.src_lang and tokenizer.tgt_lang to the right values.

In [11]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [12]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


In [13]:
max_length = tokenizer.model_max_length
print(f"max_length: {max_length}")


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

max_length: 512


In [14]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

## Fine-tuning the model with the Trainer API

In [15]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

#### Data collation

In [16]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [17]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [18]:
batch["labels"]
# our labels have been padded to the maximum length of the batch, using -100

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

In [19]:
batch["decoder_input_ids"]
# Decoder input IDs, which are shifted versions of the labels with a special token at the beginning

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

In [20]:
# labels for the first and second elements in our dataset
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]
[1211, 3, 49, 9409, 1211, 3, 29140, 817, 3124, 817, 550, 7032, 5821, 7907, 12649, 0]


In [21]:
print("Pad token ID:", tokenizer.pad_token_id)
print("Pad token:", tokenizer.pad_token)


Pad token ID: 59513
Pad token: <pad>


In [22]:
print("Collator padding token ID:", data_collator.tokenizer.pad_token_id)


Collator padding token ID: 59513


In [23]:
# Check what token ID 0 corresponds to
token_id = 0
token = tokenizer.convert_ids_to_tokens(token_id)
print(f"Token ID {token_id} corresponds to token: '{token}'")


Token ID 0 corresponds to token: '</s>'


### Metrics

In [24]:
!pip install sacrebleu

In [25]:
import evaluate

metric = evaluate.load("sacrebleu")

In [26]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [27]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [28]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [29]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

### Fine-tuning the model

In [30]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr-checkpoints",
    evaluation_strategy="no",  # we will just evaluate our model once before training and after.
    save_strategy="epoch",  # Save checkpoints at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,  # Limit the number of saved checkpoints
    num_train_epochs=10,
    predict_with_generate=True,  # needed to generate the translations
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

c:\Users\priks\anaconda3\envs\t\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
trainer.evaluate(max_length=max_length)

100%|██████████| 329/329 [17:22<00:00,  3.17s/it]


{'eval_loss': 1.7000652551651,
 'eval_model_preparation_time': 0.002,
 'eval_bleu': 38.973943774115114,
 'eval_runtime': 1051.0115,
 'eval_samples_per_second': 19.998,
 'eval_steps_per_second': 0.313}

In [32]:
trainer.train()

  1%|          | 502/59120 [01:12<1:22:01, 11.91it/s] 

{'loss': 1.3916, 'grad_norm': 7.633892059326172, 'learning_rate': 1.9831190798376187e-05, 'epoch': 0.08}


  2%|▏         | 1000/59120 [02:22<1:27:25, 11.08it/s]

{'loss': 1.2408, 'grad_norm': 4.5788655281066895, 'learning_rate': 1.9662043301759137e-05, 'epoch': 0.17}


  3%|▎         | 1500/59120 [03:24<2:37:07,  6.11it/s] 

...

100%|█████████▉| 59002/59120 [2:15:08<00:10, 10.80it/s]

{'loss': 0.5434, 'grad_norm': 2.6248016357421875, 'learning_rate': 4.769959404600812e-08, 'epoch': 9.98}


100%|██████████| 59120/59120 [2:15:28<00:00,  7.27it/s]

{'train_runtime': 8128.9827, 'train_samples_per_second': 232.692, 'train_steps_per_second': 7.273, 'train_loss': 0.7104815463735867, 'epoch': 10.0}


TrainOutput(global_step=59120, training_loss=0.7104815463735867, metrics={'train_runtime': 8128.9827, 'train_samples_per_second': 232.692, 'train_steps_per_second': 7.273, 'total_flos': 4.123593985189478e+16, 'train_loss': 0.7104815463735867, 'epoch': 10.0})

In [35]:
trainer.evaluate(max_length=max_length)

100%|██████████| 329/329 [1:40:07<00:00, 18.26s/it]  


{'eval_loss': 0.7965446710586548,
 'eval_model_preparation_time': 0.002,
 'eval_bleu': 54.95663610516514,
 'eval_runtime': 6012.2223,
 'eval_samples_per_second': 3.496,
 'eval_steps_per_second': 0.055,
 'epoch': 10.0}

In [36]:
# Saving the Model and Tokenizer
model_output_dir = "marian-finetuned-kde4-en-to-fr"

trainer.save_model(model_output_dir)

tokenizer.save_pretrained(model_output_dir)

('marian-finetuned-kde4-en-to-fr\\tokenizer_config.json',
 'marian-finetuned-kde4-en-to-fr\\special_tokens_map.json',
 'marian-finetuned-kde4-en-to-fr\\vocab.json',
 'marian-finetuned-kde4-en-to-fr\\source.spm',
 'marian-finetuned-kde4-en-to-fr\\target.spm',
 'marian-finetuned-kde4-en-to-fr\\added_tokens.json')

In [37]:
import json
import os

log_history = trainer.state.log_history

# Save the entire log_history to log_history.json
log_history_output_file = os.path.join(model_output_dir, "log_history.json")
with open(log_history_output_file, "w") as f:
    json.dump(log_history, f, indent=4)

print(f"Log history saved to {log_history_output_file}")

Log history saved to marian-finetuned-kde4-en-to-fr\log_history.json


In [40]:
from transformers import pipeline

model_checkpoint = "marian-finetuned-kde4-en-to-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'translation_text': 'Par défaut, développer les fils de discussion'}]